# From test prompts to LLM answers

This notebook implements the execution of a scenario (prompts) designed in *stc-rdf-test*.

Note : The code in the cells bellow should be added in a main function to be executed in command line.

In [1]:
import sys
print(sys.version)

3.13.2 (v3.13.2:4f8bb3947cf, Feb  4 2025, 11:51:10) [Clang 15.0.0 (clang-1500.3.9.4)]


In [2]:
import ollama
import pandas as pd
from pathlib import Path
from llms_classes import *
from prompt_classes import *
from dataset_classes import TextDataset

In [3]:
d=TextDataset()

In [4]:
d.filter_dataset_notclass()

In [5]:
p=PromptCreation()

In [6]:
res=p.prompt_creation()

In [7]:
for i in res:
    print(i)

Sei un classificatore di testi pubblicati su twitter (adesso X). Pensi che chi ha scritto la frase voglia attaccare, sostenere o essere neutrale verso la comunità LGBTQ+? Rispondi con un unica parola. 

INPUT: Meglio fascista che frocio? Sceglie fra due disgrazie. Sono contro l'omofobia, ma i froci non hanno mai mandato in Russia 67k mila ragazzi senza equipaggiamento, dei quali solo 7k tornati ma congelati, fra cui mio nonno. E' fiera di suo nonno che ha fatto questo  e .... vergogna!
LABEL: Attacca

INPUT: @Rosi33998582 @mps274 No. Questo è un insulto lesbica no LGBT sta per Lesbica Gay Bisessuali Trans usare la parola froc*o ha una connotazione dispregiativa come lo è diventata neg*o che invece prima si usava ( se sbaglio correggetemi per favore ).
LABEL: Sostiene

INPUT: {}
LABEL: 



## 2. Setup the Ollama server

Before executing this code, an Ollama server instance has to be launched in the terminal using the ```ollama serve``` command.

In [8]:
MODEL = "llama3.1"
MODEL1= "mistral"
MODEL = "qwen2.5"

In [9]:
# Init the ollama server
ollama_server = OllamaServer(ollama)

# Check models that have already been downloaded
models = ollama_server.get_models_list()
print("Available Models:", models)

# Download the model to use for experiences
models = ollama_server.download_model_if_not_exists(MODEL)

Available Models: ['qwen2.5:latest', 'deepseek-r1:7b', 'llama3.2:3b', 'gemma2:2b', 'gemma2:latest', 'phi3.5:latest', 'llama3.1:latest', 'qwen2:latest', 'phi3:latest', 'tinyllama:latest', 'qwen2:1.5b', 'qwen2:0.5b', 'llama3:latest']
qwen2.5 is already available in the server


## 3. Execute the prompts

In [10]:
# Initialize the client and server
ollama_server = OllamaServer(ollama)

# Initialize chat with a specific model
chat = OllamaChat(server=ollama_server, model=MODEL)

qwen2.5 is already available in the server


In [11]:
d.dataset_filter['TEXT']

TypeError: 'NoneType' object is not subscriptable

In [11]:
LLM_Responses = []

for i in range(len(d.dataset_filter['TEXT'])):
    
    print("################## PROMPT #####################")
    print(res.to_list()[0].format(i))
    print("################## RESPONSE #####################")
    print("Pred")
    response = chat.send_prompt(res.to_list()[0], prompt_uuid="1", use_history=False, stream=True)
    print("\n")

    print("True")
    print(d.dataset_filter['Pensi che chi ha scritto la frase voglia attaccare, sostenere o essere neutrale verso la comunità LGBTQ+?'][i])
    print("\n")
    #LLM_Responses.append(response)
    #print(response.row_text)

TypeError: 'NoneType' object is not subscriptable